In [ ]:
import requests

from pydantic import BaseModel, Field, RootModel, field_validator
import pandas as pd

youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res = requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

class Site(BaseModel):
    行政區域: str = Field(alias='sarea')
    總共數量: int = Field(alias='total')
    可借數量: int = Field(alias='available_rent_bikes')
    可還數量: int = Field(alias='available_return_bikes')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def get_name(cls, value: str) -> str:
        return '0.0' if value == '' else value

class YouBike(RootModel):
    root: list[Site]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

df = pd.DataFrame(all_sites)
df1=df.set_index(keys='行政區域')
df1_sum=df1.groupby(level='行政區域').sum()
df1_sum